In [9]:
using JuMP, Gurobi
using CSV, LinearAlgebra, DataFrames

In [37]:
#load the data and orgnize 
FL = repeat([Float64], inner=15)
dtype = append!([String], FL);

regional_EF = CSV.File("C:/Users/bourg/.julia/environments/batterySC/Li-battery-SC/data/social/new_EF_SC1.csv",header=1,delim=",", types=dtype) |> DataFrame    
capacity = CSV.File("C:/Users/bourg/.julia/environments/batterySC/Li-battery-SC/data/social/capacity1.csv",header=1,delim=",", types=dtype) |> DataFrame    
distance = CSV.File("C:/Users/bourg/.julia/environments/batterySC/Li-battery-SC/data/social/distance.csv",header=1,delim=",") |> DataFrame 
LCA_model = CSV.File("C:/Users/bourg/.julia/environments/batterySC/Li-battery-SC/data/social/LCA_model1.csv",header=1,delim=",") |> DataFrame 
D_Dsoc = CSV.File("C:/Users/bourg/.julia/environments/batterySC/Li-battery-SC/data/social/D_Dsoc1.csv",header=1,delim=",") |> DataFrame
GDP = CSV.File("C:/Users/bourg/.julia/environments/batterySC/Li-battery-SC/data/social/GDP.csv",header=1,delim=",") |> DataFrame;

In [40]:
cell_demand = 0.001*164.98*(1.369*1e6)*3           # annual demand of Li battery for tesla (1.369M EV/yr, ~2 NMC111 pack/EV, 226 kg/pack (35kwh/pack), 80~100 kWh per EV)

global_sink = 1.099e10                        # global pub (ocean) CO2 sequestration (ton/yr)
global_sink_tot = 2.236e10                  # global total (ocean+land) CO2 sequestration (ton/yr)
global_emi = 3.53e10                          # global CO2 emission (ton/yr)
es_ratio = global_sink/global_emi
es_ratio_tot = global_sink_tot/global_emi
D = D_Dsoc[!, "D"]          # national CO2 emission (ton/yr)
Dsoc = D_Dsoc[!, "Dsoc ton/yr"]           # national CO2 sink (ton/yr)

EF_trans = 1.005/10000                        # ton CO2/km*ton (The average freight truck in the U.S. emits 161.8 grams of CO2 per ton-mile)
process = LCA_model[!,"process"]
countries = capacity[!,"country"]
ncty = size(countries,1)                          # No. of countries
nproc = size(process,1);                          # No. of processes 

mkt_loc = findfirst(isequal("United States"), countries)
mkt_proc = findfirst(isequal("battery"), process)

# seperate model
cathode = 1:4
cell = 5:10
noncell = [12,13]
battery = [11,14]
scaler = LCA_model[!,"scaler"]
price = LCA_model[!,"price (usd/kg product)"]
vGDP = GDP[!,"GDP usd"];

In [16]:
LCA_model

Row,process,scaler,price (usd/kg product)
,String15,Float64,Float64
1,Li,0.383,40.0
2,Co,0.536,79.5
3,Mn,0.522,1.51
4,Ni,0.535,20.13
5,cathode,0.396,7.4
6,graphite,0.221,20.0
7,PP,0.016,1.208
8,PE,0.004,1.143
9,Cu,0.095,8.21


In [17]:
up_cath = scaler[1:4] * scaler[5] * scaler[11]
up_cell = scaler[5:10] * scaler[11]
cell_sef = scaler[11]
up_noncell = scaler[12:13] * scaler[14]
noncell_sef = scaler[14]
battery_sef = scaler[15]

1.0

In [18]:
input_amount = []
input_amount = vcat(up_cath, up_cell, cell_sef, up_noncell, noncell_sef, battery_sef) .* cell_demand
input_amount

15-element Vector{Float64}:
  74505.437384598
 104268.70610481601
 101545.269751332
 104074.17493671
 194531.168106
 108564.11149350001
   7859.845175999999
   1964.9612939999997
  46667.83073249999
  90388.21952399999
 491240.3235
   3387.8643
   2710.29144
 677572.86
 677572.86

----

In [19]:
model = Model(Gurobi.Optimizer);

Academic license - for non-commercial use only - expires 2023-11-27


In [20]:
#variables 
@variable(model, x[1:ncty, 1:nproc] >= 0)            # x[i,k] production amount of product k at location i
@variable(model, y[1:ncty, 1:ncty, 1:nproc] >= 0)   # y[i,j,k] ship product k from i to j 
@variable(model, delta[1:ncty], binary=true);

In [21]:
# capacity constraint: x[i,k] <= capacity[i][k]
for k in 1:nproc
    for i in 1:ncty
        @constraint(model, x[i,k] <= capacity[!, 2:end][i,k])
    end
end

In [22]:
# node output flow constraint
for k in 1:nproc
    for i in 1:ncty
        @constraint(model, sum(y[i,j,k] for j in 1:ncty) == x[i,k])
    end
end

In [23]:
# node output flow to market y[i,38, 15]
for i in 1:ncty
    for j in [e for e in 1:ncty if e != mkt_loc]
        @constraint(model, y[i,j,mkt_proc] == 0)
    end
end

In [24]:
for k in 1:nproc-1
    @constraint(model, sum(x[i,k] for i in 1:ncty) == input_amount[k])
end

In [25]:
# final demand constraint
@constraint(model, sum(y[i,mkt_loc,mkt_proc] for i in 1:ncty) == cell_demand);

In [26]:
# cathode LCA constraints (index=5)
for k in cathode
    for j in 1:ncty
        @constraint(model, sum(y[i,j,k] for i in 1:ncty) == x[j,5] * scaler[k])
    end
end


# cell LCA constraints (index=11)
for k in cell
    for j in 1:ncty
        @constraint(model, sum(y[i,j,k] for i in 1:ncty) == x[j,11] * scaler[k])
    end
end


# non cell LCA constraints (index=14)
for k in noncell
    for j in 1:ncty
        @constraint(model, sum(y[i,j,k] for i in 1:ncty) == x[j,14] * scaler[k])
    end
end


# battery LCA constraints (index=15)
for k in battery
    for j in 1:ncty
        @constraint(model, sum(y[i,j,k] for i in 1:ncty) == x[j,15] * scaler[k])
    end
end


In [27]:
Pemi = 0
Pincome = 0
proD = Vector{AffExpr}(undef, ncty)
proM = Vector{AffExpr}(undef, ncty)

for i in 1:ncty
    for k in 1:nproc
        Pemi += regional_EF[i,k+1] * x[i,k]
        Pincome += x[i,k] * price[k]
    end
    proD[i] = Pemi
    proM[i] = Pincome
end


Temi = 0
transD = Vector{AffExpr}(undef, ncty)
for i in 1:ncty
    for j in 1:ncty
        amount = sum(y[i,j,k] for k in 1:nproc)
        Temi += amount * distance[!, 2:end][i,j] * EF_trans
    end
    transD[i] = Temi
end

In [28]:
M = 1e15

1.0e15

In [43]:
Allo_soc = proM ./ vGDP .* Dsoc
SJ = proD + transD - Allo_soc;

In [44]:
for i in 1:ncty
    @constraint(model, SJ[i] >= (1 - delta[i]) * M)
end

In [45]:
@objective(model, Max, sum(delta));

In [46]:
JuMP.optimize!(model)

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 2235 rows, 13980 columns and 238020 nonzeros
Model fingerprint: 0x7d33165e
Variable types: 13950 continuous, 30 integer (30 binary)
Coefficient statistics:
  Matrix range     [4e-03, 1e+15]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+02, 1e+15]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Found heuristic solution: objective 30.0000000
Presolve removed 1957 rows and 12595 columns
Presolve time: 0.05s
Presolved: 278 rows, 1385 columns, 3943 nonzeros
Variable types: 1385 continuous, 0 integer (0 binary)

Explored 0 nodes (0 simplex iterations) in 0.08 seconds
Thread count was 12 (of 12 available processors)

Solution count 1: 30 

Optimal solution found (tolerance 1.00e-04)
Best objective 3.000000000000e+01, best bound 3.0000000

In [47]:
optx = copy(JuMP.value.(x))
opty = copy(JuMP.value.(y));

In [93]:
res_x = DataFrame(optx, :auto)
rename!(res_x, ["x$i" => proc for (i, proc) in enumerate(process)])
insertcols!(res_x, 1, :country => countries);

In [94]:
res_x

Row,country,Li,Co,Mn,Ni,cathode,graphite,PP,PE,Cu,Al,cell,PET,electronics,noncell,battery
,String,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,Argentina,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Australia,32705.4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Brazil,2200.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,46667.8,0.0,0.0,0.0,0.0,0.0,0.0
4,Canada,0.0,0.0,0.0,1.04074e5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2710.29,0.0,0.0
5,Chile,39000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,China,0.0,0.0,0.0,0.0,0.0,98314.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.77573e5
7,Colombia,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,Congo,0.0,1.04269e5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,Finland,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [95]:
CSV.write("/Users/bourg/Desktop/x_matrix_regional0915.csv", res_x) 

"/Users/bourg/Desktop/x_matrix_regional0915.csv"

In [49]:
using Tables
res = Tables.table(optx[1:end, 1:end]);
CSV.write("/Users/bourg/Desktop/x_matrix_regional0914.csv", res) 

"/Users/bourg/Desktop/x_matrix_regional0914.csv"

In [62]:
res_y02 = Tables.table(opty[1:end, 1:end, 02])
CSV.write("/Users/bourg/Desktop/y02_matrix.csv", res_y02) 

"/Users/bourg/Desktop/y02_matrix.csv"

In [38]:
# # 2. transpotration emission
# trans_emi = 0
# for k in 1:nproc
#     for i in 1:ncty
#         for j in 1:ncty
#             trans_emi += y[i,j,k] * distance[!, 2:end][i,j] * EF_trans;
#         end
#     end
# end

In [15]:
# Pemi = 0
# Pincome = 0
# proD = zeros(ncty)
# proM = zeros(ncty)
# for i in 1:ncty
#     for k in 1:nproc
#         Pemi += regional_EF[i, k+1] * x[i,k] 
#         Pincome += x[i,k] * price[k]
#     end
#         proD[i] = Pemi
#         proM[i] = Pincome
# end
        
   

# Temi = 0
# transD = zeros(ncty)
# for i in 1:ncty
#     for j in 1:ncty
#         amount = sum(y[i,j,k] for k in 1:nproc)
#         Temi += amount * distance[!, 2:end][i,j] * EF_trans
#     end
#     transD[i] = Temi
# end


# Allo_soc = proM ./ vGDP .* Dsoc
# SJ = proD + transD - Allo_soc
    
# M = Inf
